#Intitialization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from dateutil.parser import parse as dateparse
import time
from collections import Counter
from tqdm.notebook import tqdm
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from datetime import datetime
from dateutil.parser import parse as dateparse
import pickle
import csv

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

In [ ]:
from google.colab import drive, files
# drive.mount('/content/drive')

FILELOC = "/content/drive/My Drive/COVID_Python/"

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.TPUStrategy(tpu)
    tpu_env=True
except ValueError:
    print('Not connected to a TPU runtime.')
    tpu_env=False

Not connected to a TPU runtime.


#Dataset creation Pipelines

###Combine and pre-process patient metadata files

In [ ]:
origdf = pd.read_csv(FILELOC + "GISAID_patient_metadata_20210811.tsv", '\t')
patdf = pd.read_csv(FILELOC + 'GISAID_patient_metadata_20210812_0912.tsv', '\t')
newdf = pd.read_csv(FILELOC + 'GISAID_patient_metadata_20210913_1004_raw.tsv', '\t')
print(len(origdf), len(patdf), len(newdf))

113680 23073 18792


In [ ]:
def proc_df(mdf2, patientstatusmapfile, agemapfile):

    labelmap = {'alive' : -1,
                'asymptomatic' : 0,
                'dead' : 1,
                'hospitalized' : 1,
                'mild' : 0,
                'moderate' : 0,
                'released' : 1,
                'screening' : 0,
                'severe' : 1,
                'symptomatic' : -1,
                'unknown' : -1,
            }

    mdf2.rename(columns={'Corrected status':'Status', 'Gender':'Gender_Raw',
                        'Patient age':'Age_Raw'}, inplace=True)
    Location_Parsed = mdf2['Location'].apply(lambda x: x.split('/'))
    mdf2['Country'] = [x[1].strip() for x in Location_Parsed]
    agemapdf = pd.read_csv(agemapfile, '\t', encoding='ISO-8859-1', dtype={'Original':'str', 'Processed':'Int64'})
    agemap = dict(zip(agemapdf.Original, agemapdf.Processed))
    mdf2['Age'] = mdf2['Age_Raw'].map(agemap).fillna(-1)

    statusmapdf = pd.read_csv(patientstatusmapfile, "\t")
    catmapdf = pd.read_csv(FILELOC + "patient_status_reduced_map_20210811.csv")
    statusmap = dict(zip(statusmapdf['Original'].apply(lambda x: x.lower()),
                        statusmapdf['Map1'].apply(lambda x: x.lower())))
    catmap = dict(zip(catmapdf['Status'].apply(lambda x: x.lower()),
                    catmapdf['Category'].apply(lambda x: x.lower())))
    mdf2['Category'] = mdf2['Status'].apply(lambda x: x.lower()).map(statusmap).map(catmap)
    mdf2['Category'] = mdf2['Category'].fillna('unknown')
    mdf2['Label'] = mdf2['Category'].map(labelmap).astype(int)

    return mdf2

In [ ]:
odf = proc_df(origdf, FILELOC+'patient_status_map_20210811.tsv',
              FILELOC+'age_map_20210811.tsv')
pdf = proc_df(patdf, FILELOC+'patient_status_map_20210812_0912.tsv',
              FILELOC+'age_map_20210812_0912.tsv')
ndf = proc_df(newdf, FILELOC+'patient_status_map_20210913_1004.tsv',
              FILELOC+'age_map_20210913_1004.tsv')

In [ ]:
# !tar -xvf '/content/drive/My Drive/COVID_Python/results1001_Spike.tar.gz' -C '/content/drive/My Drive/COVID_Python/'
# seqs = pd.read_csv(FILELOC + 'results1001_Spike.csv')
# seqlist = pd.concat([origdf['Accession ID'],patdf['Accession ID'],newdf['Accession ID']],
#                     axis=0, ignore_index=True).to_frame()
# seqlist.loc[len(seqlist)] = 'EPI_ISL_402124' # add reference sequence
# print(len(seqs), len(seqlist))
# selseqs = pd.merge(left=seqs,right=seqlist,left_on='SequenceID',right_on='Accession ID')
# print(len(selseqs))
# selseqs.to_csv(FILELOC + 'results_1001_Spike_selected.csv', index=False)

In [ ]:
alldf = pd.concat([odf, pdf, ndf], ignore_index=True, axis=0)

##Create cross-validation datasets

In [ ]:
data = pd.read_csv(FILELOC + 'covid_rawseqs_crossval/covid_patient_0912_raw_sequences.csv')
data.reset_index(drop=True, inplace=True)
np.savetxt(FILELOC + 'covid_rawseqs_crossval/TrainLabels.csv', data.Label)
with open(FILELOC + 'covid_rawseqs_crossval/TrainData.csv', 'w') as f:
    csv.writer(f).writerows(data.ISM.values)

random_state_split = 32687     # for splitting
NSplits = 10  ## Number of folds
skf = KFold(n_splits = NSplits, shuffle = True, random_state = random_state_split)
fold=0
for train_index, test_index in skf.split(data):
    np.savetxt(FILELOC + 'covid_rawseqs_crossval/TrainIndex_'+str(fold)+'.csv', train_index, '%i', delimiter=',')
    np.savetxt(FILELOC + 'covid_rawseqs_crossval/TestIndex_'+str(fold)+'.csv', test_index, '%i', delimiter=',')
    fold += 1

In [ ]:
data = pd.read_csv(FILELOC + 'covid_aligned_crossval/covid_patient_0912_aligned_sequences.csv')
data.reset_index(drop=True, inplace=True)
np.savetxt(FILELOC + 'covid_aligned_crossval/TrainLabels.csv', data.Label)
with open(FILELOC + 'covid_aligned_crossval/TrainData.csv', 'w') as f:
    csv.writer(f).writerows(data.ISM.values)

random_state_split = 32687     # for splitting
NSplits = 10  ## Number of folds
skf = KFold(n_splits = NSplits, shuffle = True, random_state = random_state_split)
fold=0
for train_index, test_index in skf.split(data):
    np.savetxt(FILELOC + 'covid_aligned_crossval/TrainIndex_'+str(fold)+'.csv', train_index, '%i', delimiter=',')
    np.savetxt(FILELOC + 'covid_aligned_crossval/TestIndex_'+str(fold)+'.csv', test_index, '%i', delimiter=',')
    fold += 1

In [ ]:
data = pd.read_csv(FILELOC + 'covid_filtered_crossval/covid_patient_0912_filtered_sequences.csv')
data.reset_index(drop=True, inplace=True)
np.savetxt(FILELOC + 'covid_filtered_crossval/TrainLabels.csv', data.Label)
with open(FILELOC + 'covid_filtered_crossval/TrainData.csv', 'w') as f:
    csv.writer(f).writerows(data.ISM.values)

random_state_split = 32687     # for splitting
NSplits = 10  ## Number of folds
skf = KFold(n_splits = NSplits, shuffle = True, random_state = random_state_split)
fold=0
for train_index, test_index in skf.split(data):
    np.savetxt(FILELOC + 'covid_filtered_crossval/TrainIndex_'+str(fold)+'.csv', train_index, '%i', delimiter=',')
    np.savetxt(FILELOC + 'covid_filtered_crossval/TestIndex_'+str(fold)+'.csv', test_index, '%i', delimiter=',')
    fold += 1

##Generate patient datasets (aligned, raw, filtered)

In [ ]:
 alldf = pd.read_csv(FILELOC + 'covid_alldata_merged_align1001.csv')
# alldf = pd.read_csv(FILELOC + 'covid_alldata_merged_rawseq1001.csv')

alldf.rename(columns={'MaskedSeq':'ISM'}, inplace=True)
# alldf.rename(columns={'Spike':'ISM'}, inplace=True)

In [ ]:
print(len(alldf))
alldf.drop(alldf[alldf['ISM'].str.contains('\*')].index, inplace=True)
alldf.drop(alldf[alldf['ISM'].str.contains('X')].index, inplace=True)
print(len(alldf))

147168
105240


In [ ]:
print("Dropping invalid label values", len(alldf))
alldf.drop(alldf[alldf.Label==-1].index, inplace=True)
print(len(alldf))
alldf.reset_index(drop=True, inplace=True)

# print("Merging with sequences")
# print(len(alldf))
# alldf_rawseqs = pd.merge(left=alldf, right=seqs, left_on='Accession ID', right_on='SequenceID')
# # alldf_rawseqs = pd.merge(left=alldf, right=df, left_on='Accession ID', right_on='SequenceID')
# print(len(alldf_rawseqs))
# alldf_rawseqs.rename(columns={'MaskedSeq':'ISM'}, inplace=True)
# alldf_rawseqs.rename(columns={'Spike':'ISM'}, inplace=True)

Dropping invalid label values 105240
54081


In [ ]:
olddata = pd.read_csv(FILELOC + "patient_spike_ism_20210811.tsv", '\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
mdf2 = pd.read_csv(FILELOC + 'align0912_spike.csv')
patdf = pd.read_csv(FILELOC + 'GISAID_patient_metadata_20210812_0912.tsv', '\t')
olddata2 = pd.merge(left=mdf2[['SequenceID','MaskedSeq']], right = patdf,
                    left_on = 'SequenceID', right_on='Accession ID')
print(len(mdf2), len(patdf), len(olddata2))

22167 23073 22166


In [ ]:
print(len(alldf), len(olddata), len(olddata2))
earlydata = alldf[alldf['Accession ID'].isin(olddata['Accession ID']) | alldf['Accession ID'].isin(olddata2['Accession ID']) ]
# earlydata = alldf_rawseqs[alldf_rawseqs['Accession ID'].isin(olddata['Accession ID'])]
print(len(earlydata))

54081 104776 22166
46328


In [ ]:
# seldata = alldf.copy()
seldata = earlydata.copy()

countdata = seldata.groupby(["ISM","Label"]).size().to_frame().rename(columns={0:"Count"})
newdata = pd.DataFrame(seldata.ISM.value_counts()).rename(columns={"ISM":"Count"})
newdata['Count_Dict'] = [countdata.loc[ind].to_dict()["Count"] for ind in newdata.index]
newdata = newdata.reset_index(drop=False).rename(columns={"index":"ISM"})
label_list = np.zeros(len(newdata))
for ind in newdata.index:
    elem = newdata.Count_Dict[ind]
    if len(elem) == 2:
        label_list[ind] = newdata.Count_Dict[ind][1] / newdata.Count[ind]
    elif 0 in elem:
        label_list[ind] = 0
    elif 1 in elem:
        label_list[ind] = 1
newdata['Label'] = label_list
totalcount = np.sum(newdata.Count)
newdata['sample_weight'] = newdata['Count'].apply(lambda x: x/totalcount*10)

seldata = newdata.copy()

In [ ]:
basedata = seldata.copy()
# alldata = seldata.copy()

In [ ]:
bd = basedata[basedata.Count >= 5]
ad = alldata[~alldata.ISM.isin(bd.ISM)]
print(len(alldata), len(bd), len(ad), len(ad[ad.Count >= 5]))

# alldata.to_csv(FILELOC+"covid_patient_all_raw_sequences_1001.csv", index=False)
# bd.to_csv(FILELOC + 'covid_patient_0912_raw_sequences.csv', index=False)
# ad.to_csv(FILELOC + 'covid_patient_0912_valid1001_raw_sequences.csv', index=False)
# alldata.to_csv(FILELOC+"covid_patient_all_aligned_sequences_1001.csv", index=False)
# bd.to_csv(FILELOC + 'covid_patient_0912_aligned_sequences.csv', index=False)
# ad.to_csv(FILELOC + 'covid_patient_0912_valid1001_aligned_sequences.csv', index=False)
alldata.to_csv(FILELOC+"covid_patient_all_filtered_sequences_1001.csv", index=False)
bd.to_csv(FILELOC + 'covid_patient_0912_filtered_sequences.csv', index=False)
ad.to_csv(FILELOC + 'covid_patient_0912_valid1001_filtered_sequences.csv', index=False)

7557 716 6841 96


## B.1.640 Dataset

In [ ]:
!pip install scikit-bio
import skbio
from skbio import TabularMSA, DNA, Protein
from skbio.alignment import local_pairwise_align_ssw
!pip install biopython
from Bio.Align import substitution_matrices
import itertools

     |████████████████████████████████| 2.3 MB 5.2 MB/s 


In the code below, OMICRONFILE can be any specific dataset (this was taken from the code used to obtain the Omicron spike sequences)

In [ ]:
DIR = FILELOC + 'coronavirus_spike/'
SPIKEFILE = DIR + 'EPI_ISL_402124-S.fasta'
# OMICRONFILE = DIR + 'gisaid_hcov-19_2022_01_07_21_B.1.640.fasta'
OMICRONFILE = DIR+'gisaid_hcov-19_2022_01_28_00.fasta' # BA.2 sequences before Dec. 15, 2022

seqs = [seq for seq in skbio.io.read(OMICRONFILE, format='fasta')]
spike = DNA.read(SPIKEFILE)  # reference sequence from NCBI
align = [local_pairwise_align_ssw(DNA(seq),spike) for seq in seqs]
# align is a triplet of (alignment, score, start_end_positions)
spike_prot = ''.join([x.decode() for x in spike.translate().values])

# find aligned spike sequences without degenerates to translate
no_degenerates = [k for k in range(len(seqs)) if not align[k][0][0].has_degenerates()]
omicron_protein_seqs = [align[k][0][0].degap().translate() for k in no_degenerates]
omicron_seqs = [''.join([x.decode() for x in seq.values]) for seq in omicron_protein_seqs]
omicron_metadata = [seq.metadata['id'] for seq in omicron_protein_seqs]

# find aligned spike sequences without degenerates to translate
no_degenerates = [k for k in range(len(seqs)) if not align[k][0][0].has_degenerates()]
spikelocs = [align[k][2][0] for k in range(len(seqs))]
omicron_spike_seqs = [seqs[k][spikelocs[k][0]:spikelocs[k][1]+1] for k in no_degenerates]
omicron_protein_seqs = [DNA(seq).translate() for seq in omicron_spike_seqs]
omicron_seqs = [''.join([x.decode() for x in seq.values]) for seq in omicron_protein_seqs]
omicron_metadata = [seq.metadata['id'] for seq in omicron_protein_seqs]

In [ ]:
omicrondf = pd.DataFrame.from_dict({'Metadata':omicron_metadata})
def f(x):
    try:
        return x.split('|')[1]
    except:
        return x
omicrondf['SeqID'] = omicrondf.Metadata.apply(f)
omicrondf['Raw'] = omicron_seqs
omicrondf['seqlen'] = omicrondf['Raw'].apply(len)

# # generate modified aligned sequences where there are insertions relative to
# # reference sequence, i.e., where the inserted residues are deleted

blosum62 = substitution_matrices.load("BLOSUM62")
def f(x):
    alignment,score,startend = local_pairwise_align_ssw(Protein(x),
                                                        Protein(spike_prot),
                                                        substitution_matrix=blosum62)
    seqalign = ''.join([x.decode() for x in alignment[0].values])
    refalign = ''.join([x.decode() for x in alignment[1].values])
    return (seqalign, refalign)

alignments = omicrondf['Raw'].apply(f).tolist()
omicrondf['seqalign'] = [a[0] for a in alignments]
omicrondf['refalign'] = [a[1] for a in alignments]

def f(x):
    seqalign, refalign = x
    try:
        if '-' in seqalign: # gap in aligned reference sequence indicates insertion
            gaps = [0 if c == '-' else 1 for c in refalign]
            return ('').join(itertools.compress(seqalign, gaps))
        else:
            return seqalign
    except:
        print('Error')
        return np.nan

# also remove stopcodons
MAXLEN = 1273
def remstop(x):
    if x[-1] == '*' and len(x) > MAXLEN:
        return x[:-1]
    else:
        return x
omicrondf['Align'] = omicrondf[['seqalign','refalign']].apply(f, axis=1).apply(remstop)

# omicrondf.to_csv(FILELOC + 'B1640_spike_20220107.csv',index=False)
omicrondf.to_csv(FILELOC + 'omicron_spike_BA2_20220127.csv',index=False)
print(len(omicrondf))

20


In [ ]:
data = pd.read_csv(FILELOC + 'covid_newpatient_crossval/covid_newpatient_alldata_20210930.csv')
data.reset_index(drop=True, inplace=True)

In [ ]:
train_index_list = []; test_index_list = []
train_labels_list = []; test_labels_list = []
train_predict_list = []; test_predict_list = []

for fold in tqdm(range(10)):
    train_index = np.loadtxt(FILELOC + 'covid_newpatient_crossval/TrainIndex_'+str(fold)+'.csv', dtype='i')
    test_index = np.loadtxt(FILELOC + 'covid_newpatient_crossval/TestIndex_'+str(fold)+'.csv', dtype='i')
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_labels_list.append(data.loc[train_index]['Label'].values)
    test_labels_list.append(data.loc[test_index]['Label'].values)
    pred = np.loadtxt(FILELOC + 'covid_newpatient_output/covid_patient_crossval_20211001_predict_'+str(fold)+'.csv')
    train_predict_list.append(pred[train_index])
    test_predict_list.append(pred[test_index])

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
for fold in range(10):
    print(fold,
          mean_squared_error(train_labels_list[fold], train_predict_list[fold]),
          np.sqrt(mean_squared_error(test_labels_list[fold], test_predict_list[fold])),
          np.sqrt(mean_squared_error(test_labels_list[fold], np.mean(train_labels_list[fold])*np.ones(len(test_labels_list[fold]))))
          )

0 0.014638042230325044 0.3139640916609181 0.33004959332636913
1 0.014895349535692779 0.34497391324343424 0.2971476132486225
2 0.012205553933830781 0.3441414695162451 0.3306383144403651
3 0.009035614463175728 0.3467450902640182 0.3377550571062478
4 0.01512524239340003 0.30797196999688636 0.3456063164946015
5 0.015713130225866217 0.33105579123557455 0.3430189319652821
6 0.011228193564955096 0.33255374081796013 0.363040634449386
7 0.013227983584382522 0.3356880046993152 0.359428105632646
8 0.013675567825118714 0.3097147075583797 0.3265607025365889
9 0.014458072489585418 0.31494888333001947 0.3291354305407129


In [ ]:
print('Mean RMSE using Attention NN model =',
      np.mean(np.sqrt(mean_squared_error(test_labels_list[fold], test_predict_list[fold]))))
print('Mean RMSE guessing average of train data =',
      np.mean(np.sqrt(mean_squared_error(test_labels_list[fold], np.mean(train_labels_list[fold])*np.ones(len(test_labels_list[fold]))))))

Mean RMSE using Attention NN model = 0.31494888333001947
Mean RMSE guessing average of train data = 0.3291354305407129
